# Set parameters

In [0]:
# Parameters

NUM_TOPIC = 6
# set the number of topics here (from 5 to 10)

MIN_COUNTS = 5
MAX_COUNTS = 150
# words with count < MIN_COUNTS and count > MAX_COUNTS will be removed

MIN_LENGTH = 15
# minimum document length (number of words)after preprocessing

# half the size of the context around a word
HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model and dataset

In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from tqdm import tqdm
import nltk
import string
import spacy
from gensim import corpora, models
import sys
sys.path.append('..')

In [0]:
# Load NLP model
nlp = spacy.load('en')

# Load dataset
docs = [(i, doc) for i, doc in enumerate(sentences)]

# Preprocess dataset and create windows

In [0]:
# Preprocess dataset
encoded_docs, decoder, word_counts = preprocess(docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS)

100%|██████████| 235/235 [00:22<00:00, 10.68it/s]

number of removed short documents: 3
total number of tokens: 58294
number of tokens to be removed: 14897
number of additionally removed short documents: 1
total number of tokens: 43384

minimum word count number: 5
this number can be less than MIN_COUNTS because of document removal


In [0]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [0]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')

231it [00:00, 4403.78it/s]


In [0]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [0]:
# number of windows (equals to the total number of tokens)
data.shape[0]

43384

# Get unigram distribution

In [0]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

# Prepare word vectors

In [0]:
%%time
vocab_size = len(decoder)
embedding_dim = 50

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
print(len(texts))
model = models.Word2Vec(texts, size=embedding_dim, window=5, workers=4, min_count=1, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
    word_vectors[i] = model.wv[str(i)]

231
CPU times: user 1min 29s, sys: 117 ms, total: 1min 29s
Wall time: 46.9 s


In [0]:
# number of unique words
vocab_size

2362

# Prepare initialization for document weights

In [0]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [0]:
%%time
n_topics = NUM_TOPIC
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

CPU times: user 476 ms, sys: 2.02 ms, total: 478 ms
Wall time: 478 ms


In [0]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i + 1, ':', ' '.join([t for t, _ in topics]))

topic 1 : law right news look thing republican need public war america
topic 2 : get justice work take candidate presidential america ask republican house
topic 3 : million police pay presidential american story take call house find
topic 4 : house vote korea police news thing million north republican try
topic 5 : vote early story way family city voter police work america
topic 6 : life black take ask find work americans family right thing


In [0]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

100%|██████████| 231/231 [00:00<00:00, 1088.17it/s]


# Save data

In [0]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)